<a href="https://colab.research.google.com/github/Sawaiz8/Animal-Classification/blob/Transfer-learning-VGG16-on-animal-classification/Transfer_learning_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf


In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d utkarshsaxenadn/animal-image-classification-dataset

100% 1.60G/1.61G [00:09<00:00, 226MB/s]
100% 1.61G/1.61G [00:09<00:00, 189MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/animal-image-classification-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
tf.__version__

'2.9.2'

In [ ]:
print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from tensorflow import keras
from keras import utils
from keras.applications.vgg16 import VGG16
from keras.layers import Dense,Flatten

In [ ]:
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (150,150,3)
)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
from keras import Sequential
model = Sequential()

In [ ]:
model.add(conv_base)
model.add(Flatten())
model.add(Dense(256,activation = 'relu'))
model.add(Dense(10, activation = 'softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,814,666
Trainable params: 16,814,666
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Turn conv base trainable to false
conv_base.trainable = False

In [ ]:
model.summary() #check trainable parameter now

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 16,814,666
Trainable params: 2,099,978
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
import os
from PIL import Image

lst = ['Cat', 'Cow','Dog', 'Elephant', 'Gorilla', 'Hippo', 'Monkey', 'Panda', 'Tiger', 'Zebra']
for i in range(10):
  # Set the directory where the images are stored
  directory = '/content/Animal-Data-V2/Data-V2/Training Data/' + lst[i]
  # Iterate through all the files in the directory
  for filename in os.listdir(directory):
    # Check if the file is an image
    if not (filename.endswith(".jpeg")):
      # Open the image file
      with Image.open(os.path.join(directory, filename)) as im:
        im = im.convert('RGB')
        # Convert the image to PNG format
        im.save(os.path.join(directory, filename.rsplit(".", 1)[0] + ".jpeg"))
        # Delete the original image
        os.remove(os.path.join(directory, filename))

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/Animal-Data-V2/Data-V2/Training Data',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)

Found 19598 files belonging to 10 classes.


In [ ]:
validation_ds = keras.utils.image_dataset_from_directory(
    directory = '/content/Animal-Data-V2/Data-V2/Validation Data',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)

Found 838 files belonging to 10 classes.


In [ ]:
def process(image,label):
    image  = tf.cast(image/255, tf.float32)
    return image, label

In [ ]:
train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
model.compile(optimizer = 'adam', loss= 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
with tf.device('/gpu:0'):
    history = model.fit(train_ds, epochs = 4, validation_data = validation_ds)

Epoch 1/4
613/613 [==============================] - 56s 91ms/step - loss: 0.0969 - accuracy: 0.9691 - val_loss: 0.9447 - val_accuracy: 0.8067
Epoch 2/4
613/613 [==============================] - 57s 93ms/step - loss: 0.0903 - accuracy: 0.9704 - val_loss: 1.3900 - val_accuracy: 0.7530
Epoch 3/4
613/613 [==============================] - 55s 89ms/step - loss: 0.0788 - accuracy: 0.9736 - val_loss: 1.1165 - val_accuracy: 0.7828
Epoch 4/4
613/613 [==============================] - 56s 91ms/step - loss: 0.0689 - accuracy: 0.9774 - val_loss: 1.2181 - val_accuracy: 0.7947


In [ ]:
import os
from PIL import Image

lst = ['Cat', 'Cow','Dog', 'Elephant', 'Gorilla', 'Hippo', 'Monkey', 'Panda', 'Tiger', 'Zebra']
for i in range(10)
  # Set the directory where the images are stored
  directory = 'Animal-Data-V2/Data-V2/Validation Data/' + lst[i]
  # Iterate through all the files in the directory
  for filename in os.listdir(directory):
    # Check if the file is an image
    if not (filename.endswith(".jpeg")):
      # Open the image file
      with Image.open(os.path.join(directory, filename)) as im:
        im = im.convert('RGB')
        # Convert the image to PNG format
        im.save(os.path.join(directory, filename.rsplit(".", 1)[0] + ".jpeg"))
        # Delete the original image
        os.remove(os.path.join(directory, filename))

In [ ]:
from pathlib import Path
import imghdr

data_dir = "Animal-Data-V2/Data-V2/Validation Data/Cat"
image_extensions = [".png", ".jpg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

In [ ]:
from pathlib import Path
import imghdr

lst = ['Cat', 'Cow','Dog', 'Elephant', 'Gorilla', 'Hippo', 'Monkey', 'Panda', 'Tiger', 'Zebra']
for i in range(10):
    img_link=list(Path("Animal-Data-V2/Data-V2/Training Data/" + lst[i]).glob(r'**/*.jpg'))

    count_num=0
    for lnk in img_link:
        binary_img=open(lnk,'rb')
        find_img=tf.compat.as_bytes('JFIF') in binary_img.peek(10)#The JFIF is a JPEG File Interchange Format (JFIF). It is a standard which we gauge if an image is corrupt or substandard
        if not find_img:
            count_num+=1
            os.remove(str(lnk))
    print('Total %d pcs image delete from Dataset' % count_num)
    #this should help you delete the bad encoded

Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
Total 0 pcs image delete from Dataset
